In [2]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [3]:
!pip install youtube-transcript-api

In [4]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.0 MB/s eta 0:00:00


In [5]:
import yt_dlp
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
import pandas as pd
from tqdm.auto import tqdm
import yt_dlp
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter

def preprocess_video(video_id):
    ydl_opts = {'skip_download': True}
    video_data = []

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(f"https://www.youtube.com/watch?v={video_id}", download=False)

    transcript = YouTubeTranscriptApi.get_transcript(video_id)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)

    combined_text = ""
    combined_start_time = 0
    combined_duration = 0

    with tqdm(total=len(transcript), desc=f"Processing video {video_id}", unit="entry") as pbar:
        for i, entry in enumerate(transcript):
            combined_text += entry['text'] + " "
            if i == 0:
                combined_start_time = entry['start']
            combined_duration += entry['duration']

            if combined_duration >= 90 or i == len(transcript) - 1:
                chunk_texts = text_splitter.split_text(combined_text)
                for j, text in enumerate(chunk_texts):
                    chunk_id = f"{video_id}_{i}_{j}"
                    video_data.append({
                        'chunk_id': chunk_id,
                        'video_id': video_id,
                        'title': info['title'],
                        'channel': info['uploader'],
                        'description': info['description'],
                        'duration': info['duration'],
                        'start_time': combined_start_time,
                        'end_time': combined_start_time + combined_duration,
                        'source': f"https://www.youtube.com/watch?v={video_id}&t={int(combined_start_time)}",
                        'chunk_text': text
                    })
                combined_text = ""
                combined_start_time = 0
                combined_duration = 0
            pbar.update(1)

    return video_data

In [8]:
import re

def extract_youtube_id(url):
    # Regular expression for YouTube ID extraction
    pattern = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    match = re.search(pattern, url)
    return match.group(1) if match else None

In [9]:
# Get YouTube URL from user input
youtube_url = input("Enter YouTube URL: ")

Enter YouTube URL: https://www.youtube.com/watch?v=SJOjpprbfeE&ab_channel=AnneHildeVassb%C3%B8Hagen


In [10]:
# Extract video ID
video_id = extract_youtube_id(youtube_url)

# Display the result
if video_id:
    print(f'Video ID: {video_id}')
else:
    print('Invalid YouTube URL')

Video ID: SJOjpprbfeE


In [11]:
import pandas as pd

In [12]:
# Process each video and append the data to the list
video_data = preprocess_video(video_id)

# Convert the collected data into a DataFrame
df = pd.DataFrame(video_data)

# Display the final DataFrame
print(df)

[youtube] Extracting URL: https://www.youtube.com/watch?v=SJOjpprbfeE
[youtube] SJOjpprbfeE: Downloading webpage
[youtube] SJOjpprbfeE: Downloading ios player API JSON
[youtube] SJOjpprbfeE: Downloading player 8d9f6215
[youtube] SJOjpprbfeE: Downloading m3u8 information


Processing video SJOjpprbfeE:   0%|          | 0/69 [00:00<?, ?entry/s]

           chunk_id     video_id  \
0  SJOjpprbfeE_26_0  SJOjpprbfeE   
1  SJOjpprbfeE_47_0  SJOjpprbfeE   
2  SJOjpprbfeE_68_0  SJOjpprbfeE   

                                               title                  channel  \
0  Alfred & Shadow - A short story about emotions...  Anne Hilde Vassbø Hagen   
1  Alfred & Shadow - A short story about emotions...  Anne Hilde Vassbø Hagen   
2  Alfred & Shadow - A short story about emotions...  Anne Hilde Vassbø Hagen   

                                         description  duration  start_time  \
0  Alfred is in love. He is also angry, shameful,...       423        23.0   
1  Alfred is in love. He is also angry, shameful,...       423         0.0   
2  Alfred is in love. He is also angry, shameful,...       423         0.0   

   end_time                                            source  \
0    113.27  https://www.youtube.com/watch?v=SJOjpprbfeE&t=23   
1     93.10   https://www.youtube.com/watch?v=SJOjpprbfeE&t=0   
2     83.27   https://

In [13]:
import os
from google.colab import userdata

In [14]:
OPENAI_API_KEY  = userdata.get('IronAPI')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HuggingFace')
PINECONE_API_KEY = userdata.get('Pinecone')

In [15]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 3.8 MB/s eta 0:00:00


In [16]:
#import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = PINECONE_API_KEY

# configure client
pc = Pinecone(api_key=api_key)

In [17]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [18]:
index_name = 'question-answering-up'

In [19]:
import time
# check if index already exists
if index_name not in pc.list_indexes().names():
    # create the index
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        spec=spec
    )
    print(f"Index '{index_name}' created.")

    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

    print(f"Index '{index_name}' is ready.")
else:
    print(f"Index '{index_name}' already exists.")

# connect to the index
index = pc.Index(index_name)

Index 'question-answering-up' already exists.


In [20]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [21]:
#Intialize retriever
import torch
from sentence_transformers import SentenceTransformer

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = 'flax-sentence-embeddings/all_datasets_v3_mpnet-base'
retriever = SentenceTransformer(model_name, device=device) #load the retriever model from HuggingFace. Use the flax-sentence-embeddings/all_datasets_v3_mpnet-base model
retriever

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [25]:
batch_size = 64

In [26]:
# Function to convert numpy types to Python types
def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

In [27]:
import numpy as np

In [28]:
#To Pinecone
for i in tqdm(range(0, len(df), batch_size), desc="Upserting batches to Pinecone"):
    # Find end of batch
    i_end = min(i + batch_size, len(df))

    # Extract batch
    batch = df.iloc[i:i_end]

    # Generate embeddings for batch
    emb = retriever.encode(batch['chunk_text'].tolist()).tolist()

    # Prepare the upsert data
    upsert_data = []
    for j, vec in enumerate(emb):
        metadata = {
            'chunk_id': str(batch['chunk_id'].iloc[j]),
            'video_id': convert_to_serializable(batch['video_id'].iloc[j]),
            'title': batch['title'].iloc[j],
            'channel': batch['channel'].iloc[j],
            'description': batch['description'].iloc[j],
            'duration': convert_to_serializable(batch['duration'].iloc[j]),
            'start_time': convert_to_serializable(batch['start_time'].iloc[j]),
            'end_time': convert_to_serializable(batch['end_time'].iloc[j]),
            'source': batch['source'].iloc[j],
            'chunk_text': batch['chunk_text'].iloc[j]
        }
        upsert_data.append((str(batch['chunk_id'].iloc[j]), vec, metadata))

    # Upsert to Pinecone
    try:
        index.upsert(vectors=upsert_data)
    except Exception as e:
        print(f"Error during upsert: {e}")
        continue

print("All batches processed and upserted to Pinecone.")

# Check that we have all vectors in index
print(index.describe_index_stats())

Upserting batches to Pinecone:   0%|          | 0/1 [00:00<?, ?it/s]

All batches processed and upserted to Pinecone.
{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4}},
 'total_vector_count': 4}


In [29]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00


In [30]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

In [31]:
#Initialize Generator
#from transformers import BertTokenizer, BertForQuestionAnswering

# Load BERT tokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Load BERT model for question answering
#generator = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad').to(device)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_name= 'vblagoje/bart_lfqa'
# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained(model_name)
generator = BartForConditionalGeneration.from_pretrained(model_name).to(device)

In [64]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode(query).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
    return xc

In [126]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['chunk_text']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

In [65]:
#from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')

def format_query(query, context, max_length=1024):
    # Extract passage_text from Pinecone search result and add the <P> tag
    context_passages = [f"<P> {m['metadata']['chunk_text']}" for m in context]

    # Concatenate the query and initial context
    formatted_query = f"question: {query} context: "

    # Tokenize the formatted query to get its length
    query_tokens = tokenizer.encode(formatted_query)

    # Calculate remaining tokens for context
    remaining_tokens = max_length - len(query_tokens) - 1  # -1 for safety

    # Add context passages until we reach the token limit
    for passage in context_passages:
        passage_tokens = tokenizer.encode(passage)
        if len(passage_tokens) <= remaining_tokens:
            formatted_query += passage + " "
            remaining_tokens -= len(passage_tokens)
        else:
            break  # Stop adding context if we've reached the limit

    return formatted_query.strip()

In [53]:
query = "What are emotions and how do they work?"
result = query_pinecone(query, top_k=1)
result

{'matches': [{'id': 'SJOjpprbfeE_26_0',
              'metadata': {'channel': 'Anne Hilde Vassbø Hagen',
                           'chunk_id': 'SJOjpprbfeE_26_0',
                           'chunk_text': 'Emotions are often feared and '
                                         'avoided, because they are so '
                                         'painful. So, why focus on emotions? '
                                         'Because emotions give you important '
                                         'information. And tell you whether '
                                         'things are going your way. Emotions '
                                         'help you to survive. By providing an '
                                         'efficient, automatic way of '
                                         'respondingrapidly to important '
                                         'situations. Emotions help you to '
                                         'relate to other people a

In [54]:
from pprint import pprint

In [55]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('question: What are emotions and how do they work? context: <P> Emotions are '
 'often feared and avoided, because they are so painful. So, why focus on '
 'emotions? Because emotions give you important information. And tell you '
 'whether things are going your way. Emotions help you to survive. By '
 'providing an efficient, automatic way of respondingrapidly to important '
 'situations. Emotions help you to relate to other people and the expectations '
 'and demands of life. Emotions tells you what you need.\n'
 'When you know what you need, you can take action to meet those needs. '
 'Emotions also communicate what you feel. All your negative experiences can '
 'turn into\n'
 'a big, black and scary feeling inside You learned many of your emotional '
 'reactions long ago.  Your black shadow will therefor sometimes resemble a '
 'person who hurt you in the past. First you need to know something about what '
 'your emotions are. You have different basic emotions,\n'
 'some of which 

In [99]:
pipe = pipeline(
    "text2text-generation",
    model=generator,
    tokenizer=tokenizer,
    #max_length=min(max_model_length, 512),
    max_length=300,
    num_beams=2,
    temperature=0.7,
    top_k=100,
    top_p=0.9,
    do_sample=True,
    early_stopping=True,
    no_repeat_ngram_size=3,
    device=device
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [100]:
memory = ConversationBufferWindowMemory(k=3)
conversation = ConversationChain(
    llm=local_llm,
    memory=memory,
    verbose=True
)

In [124]:
def generate_answer(query):
    max_length = 100
    context = query_pinecone(query, top_k=1)["matches"]
    formatted_query = format_query(query, context, max_length)
    response = conversation.predict(input=formatted_query)
    cleaned_response = response.replace("/r/Ask", "")
    return pprint(cleaned_response)

In [125]:
conversation.memory.clear()

In [126]:
generate_answer(query)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: question: question: What are emotions and how do they work? context: <P> Emotions are often feared and avoided, because they are so painful. So, why focus on emotions? Because emotions give you important information. And tell you whether things are going your way. Emotions help you to survive. By providing an efficient, automatic way of respondingrapidly to important situations. Emotions help you to relate to other people and the expectations and demands of life. Emotions tells you what you need.
When you know what you need, you can take action to meet those needs. Emotions also communicate what you feel. All your negative experiences can turn into
a big, black

In [127]:
query1 = "Why are emotions avoided?"

In [128]:
generate_answer(query1)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: question: question: What are emotions and how do they work? context: <P> Emotions are often feared and avoided, because they are so painful. So, why focus on emotions? Because emotions give you important information. And tell you whether things are going your way. Emotions help you to survive. By providing an efficient, automatic way of respondingrapidly to important situations. Emotions help you to relate to other people and the expectations and demands of life. Emotions tells you what you need.
When you know what you need, you can take action to meet those needs. Emotions also communicate what you feel. All your negative experiences can turn into
a big, black 

In [131]:
query2 = "Why are emotions often avoided?"

In [132]:
generate_answer(query2)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: question: question: What are emotions and how do they work? context: <P> Emotions are often feared and avoided, because they are so painful. So, why focus on emotions? Because emotions give you important information. And tell you whether things are going your way. Emotions help you to survive. By providing an efficient, automatic way of respondingrapidly to important situations. Emotions help you to relate to other people and the expectations and demands of life. Emotions tells you what you need.
When you know what you need, you can take action to meet those needs. Emotions also communicate what you feel. All your negative experiences can turn into
a big, black 

In [133]:
query3 = "How should i handle my emotions?"

In [134]:
generate_answer(query3)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: question: Why are emotions avoided? context:
AI: It's not a question, it's a statement. It's not an argument, it is a statement, and it's an assertion. If you were to ask it a question like "What is the difference between the feeling of anger and happiness?" it would say "I don't know."
Human: question: Why shouldn't emptions be avoided? context:
AI: It's not a question, it's a statement. It's not an argument, it is a statement, and it's an assertion.
Human: question: Why are emotions often avoided? context:
AI: Emotions are a part of your brain. They are very important for survival. They help you remember what happened, how you felt, what you did, and how you f